## Cryptocurrency Research

**Context:** Before attempting to develop cryptocurrency trading strategies, I want to do some quantitative analysis of historical returns, volatility, return distributions, etc.

_Author: Matthew Garton_  
_June 8th, 2021_

In [1]:
import os
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from utils.data_ingestion import get_crypto_prices

In [ ]:
cryptos = ['BTC', 'ETH', 'XRP', 'LTC']

cc_data = {c: get_crypto_prices(c, 'USD', source='av') for c in cryptos}

In [ ]:
BTC = cc_data['BTC']['4a. close (USD)']
ETH = cc_data['ETH']['4a. close (USD)']
XRP = cc_data['XRP']['4a. close (USD)']
LTC = cc_data['LTC']['4a. close (USD)']
XMR = cc_data['XMR']['4a. close (USD)']

In [ ]:
coins = pd.concat([BTC, ETH, XRP, LTC, XMR], axis=1)

In [ ]:
coins.columns = cryptos

In [ ]:
coins.drop(columns=['XMR'], inplace=True)

Not enough data for XMR

In [ ]:
daily_ret = coins.pct_change().dropna()

In [ ]:
corrmatrix = daily_ret.corr()

In [ ]:
corrmatrix

In [ ]:
daily_ret['BTC'].plot(kind='hist', bins=50, figsize=(15, 9), title='Distribution of BTC Daily Returns (2018-Present)');

In [ ]:
daily_ret['ETH'].plot(kind='hist', color='r', bins=50, figsize=(15, 9), title='Distribution of ETH Daily Returns (2018-Present)');

In [ ]:
from scipy.stats import skew, kurtosis
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=4, sort_dicts=False)

In [ ]:
for c in cryptos:
    data = daily_ret[c]
    res = {
        c: {
            'Mean: ': np.round(np.mean(data), 4),
            'Standard Deviation: ': np.round(np.std(data), 4),
            'Skew': np.round(skew(data), 4),
            'Kurtosis': np.round(kurtosis(data), 4)
        } 
    }
    pp.pprint(res)

In [ ]:
daily_ret['XRP'].plot(kind='hist', color='k', bins=50, figsize=(15, 9), title='Distribution of XRP Daily Returns (2018-Present)');

In [ ]:
sns.pairplot(daily_ret);

In [ ]:
daily_ret.plot(figsize=(15,9), title='Daily Returns of Major Cryptos');

### Alternative Data Source: CoinAPI

**First Implementation: Getting Historical Exchange Rate Data**

In [ ]:
params = {
    'asset_id_base': 'BTC',
    'asset_id_quote': 'USD',
    'period_id': '1DAY',
    'time_start': '2020-01-01T00:00:00',
    'time_end': '2021-06-04T00:00:00'
}

def generate_request(params):
    url = 'https://rest.coinapi.io/v1/exchangerate/BTC/USD/history?period_id=1DAY&time_start=2010-01-01T00:00:00&time_end=2021-06-04T00:00:00&limit=100000'
    headers = {'X-CoinAPI-Key' : os.getenv('COINAPI_KEY')}
    return requests.get(url, headers=headers)

In [ ]:
res = generate_request(params)

In [ ]:
df = pd.DataFrame(res.json())

In [ ]:
df['rate_low'].plot(figsize=(20,14), title='BTCUSD 2010-Present');

In [ ]:
df.head()

In [ ]:
btc = df[['time_close', 'rate_open', 'rate_high', 'rate_low', 'rate_close']]

In [ ]:
btc.set_index()

**Second Implementation: Getting Options Data**

First, explore the scope of data available from CoinAPI. How many unique underlyings, 

In [5]:
url = 'https://rest.coinapi.io/v1/symbols'
headers = {'X-CoinAPI-Key' : os.getenv('COINAPI_KEY')}
response = requests.get(url, headers=headers)

In [11]:
opts = [i for i in response.json() if i['symbol_type'] == 'OPTION']

In [15]:
bitops = [j for j in opts if j['asset_id_base'] == 'BTC']

In [19]:
expos = [o['option_expiration_time'] for o in bitops]

In [22]:
unique_expos = set(expos)

In [24]:
len(unique_expos)

768

In [27]:
unique_exchanges = set([o['exchange_id'] for o in bitops])

In [28]:
unique_exchanges

{'BINANCEOPTV', 'BINANCEOPTVUAT', 'COINEX', 'DERIBIT', 'HUOBIFTS', 'OKEX'}

In [31]:
bino = [o for o in bitops if o['exchange_id'] == 'BINANCEOPTV']

In [32]:
bino

[{'symbol_id': 'BINANCEOPTV_OPT_BTC_USDT_210528_70000_C',
  'exchange_id': 'BINANCEOPTV',
  'symbol_type': 'OPTION',
  'asset_id_base': 'BTC',
  'asset_id_quote': 'USDT',
  'option_type_is_call': True,
  'option_strike_price': 70000.0,
  'option_contract_unit': 1.0,
  'option_exercise_style': 'EUROPEAN',
  'option_expiration_time': '2021-05-28T08:00:00.0000000Z',
  'data_start': '2021-04-16',
  'data_end': '2021-06-12',
  'data_quote_start': '2021-04-16T08:03:33.7310000Z',
  'data_quote_end': '2021-06-12T14:30:01.6933333Z',
  'data_trade_start': '2021-04-24T15:33:22.5790000Z',
  'data_trade_end': '2021-06-12T14:30:04.9600000Z',
  'volume_1mth': 459.3237,
  'volume_1mth_usd': 18785406.67,
  'symbol_id_exchange': 'BTC-210528-70000-C',
  'asset_id_base_exchange': 'BTC',
  'asset_id_quote_exchange': 'USDT',
  'price_precision': 0.01,
  'size_precision': 0.0001},
 {'symbol_id': 'BINANCEOPTV_OPT_BTC_USDT_210528_85000_C',
  'exchange_id': 'BINANCEOPTV',
  'symbol_type': 'OPTION',
  'asset_id_

**Getting Some Option Data**

In [37]:
# get options data
url = 'https://rest.coinapi.io/v1/ohlcv/BINANCEOPTV_OPT_BTC_USDT_210528_55000_C/history?period_id=1DAY&time_start=2021-04-16T00:00:00'
headers = {'X-CoinAPI-Key' : os.getenv('COINAPI_KEY')}
res = requests.get(url, headers=headers)

In [41]:
data = res.json()

In [42]:
len(data)

25

In [43]:
df = pd.DataFrame(data)

In [44]:
df_format = 

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,2021-04-20T00:00:00.0000000Z,2021-04-21T00:00:00.0000000Z,2021-04-20T06:46:06.8030000Z,2021-04-20T08:20:28.9870000Z,6024.02,6367.74,4572.33,5322.21,15.9825,234
1,2021-04-22T00:00:00.0000000Z,2021-04-23T00:00:00.0000000Z,2021-04-22T05:01:30.1050000Z,2021-04-22T08:50:29.4300000Z,6024.02,6367.74,4242.32,4242.32,4.1182,63
2,2021-04-23T00:00:00.0000000Z,2021-04-24T00:00:00.0000000Z,2021-04-23T01:08:19.1760000Z,2021-04-23T02:18:18.2680000Z,6024.02,6367.74,3556.67,3566.56,2.3069,44
3,2021-04-24T00:00:00.0000000Z,2021-04-25T00:00:00.0000000Z,2021-04-24T12:28:04.1770000Z,2021-04-24T12:28:04.1770000Z,6024.02,6367.74,2764.99,2998.96,4.0394,81
4,2021-04-25T00:00:00.0000000Z,2021-04-26T00:00:00.0000000Z,2021-04-25T13:41:08.4660000Z,2021-04-25T17:56:04.5370000Z,6024.02,6367.74,2764.99,3312.17,4.6965,91
5,2021-04-27T00:00:00.0000000Z,2021-04-28T00:00:00.0000000Z,2021-04-27T00:16:31.0200000Z,2021-04-27T09:59:16.0800000Z,5332.52,6367.74,2595.82,4341.31,44.4747,900
6,2021-04-28T00:00:00.0000000Z,2021-04-29T00:00:00.0000000Z,2021-04-28T19:51:17.5830000Z,2021-04-28T22:56:39.3930000Z,5322.21,5600.00,2595.82,4210.05,24.5022,500
7,2021-04-29T00:00:00.0000000Z,2021-04-30T00:00:00.0000000Z,2021-04-29T03:15:27.5780000Z,2021-04-29T03:15:27.5780000Z,5549.08,5549.08,2595.82,4034.00,4.6801,100
8,2021-04-30T00:00:00.0000000Z,2021-05-01T00:00:00.0000000Z,2021-04-30T22:57:03.5510000Z,2021-04-30T23:39:35.2430000Z,3566.56,5800.00,2595.82,5800.00,27.4420,600
9,2021-05-01T00:00:00.0000000Z,2021-05-02T00:00:00.0000000Z,2021-05-01T00:54:42.2100000Z,2021-05-01T01:16:01.5660000Z,2905.29,5800.00,2595.82,5719.69,8.8325,200
